In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# This will reload imports before executing code, allowing you to easily change contents of custom scripts
%load_ext autoreload
%autoreload 2

# Metric box classifier & synthetic data: batch plotting

**25.08.2020**

import numpy as np
import pandas as pd

In [4]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
sys.path.append('/home/sandm/Notebooks/stay_classification/src/')

In [6]:
from synthetic_data.trajectory_class import get_pickle_trajectory
from synthetic_data.trajectory import get_stay_segs, get_adjusted_stays
from stay_classification.metric_box_classifier.metric_box_classifier import stay_classifier_testing
from helper__metric_box__explore import eval_synth_data
from synthetic_data.plotting import add_plot_trajectory, add_plot_seg_boxes, add_plot_cluster_boxplots

# Start here

In [7]:
dsec = 1/3600.0
t_total = np.arange(0,24,dsec)

In [8]:
time_thresh = 1/6
dist_thresh=0.25

In [15]:
nr_stays = 3

In [16]:
data_dir = f"./testdata_training_set__canonical_{nr_stays}stays/"

data_dir_plots = data_dir + "plots/"

try:
    os.makedirs(data_dir_plots)
except OSError as e:
    print(e)
    
try:
    os.makedirs(data_dir_plots + "bad/")
except OSError as e:
    print(e)

try:
    os.makedirs(data_dir_plots + "good/")
except OSError as e:
    print(e)

[Errno 17] File exists: './testdata_training_set__canonical_3stays/plots/'


In [21]:
lens = []
precs = []
recs = []

lens3 = []
precs3 = []
recs3 = []

bad_list = []

precrec_limit = 0.80

ii = 0

length_criterion_break = False
iqr_trim = False
verbose = False

total = 1000
tots = 800 
for ii in range(tots, tots+200):
    
    # Load the data
    trajectory_tag = f"trajectory{ii}_{nr_stays}stays"    
    path_to_file =  data_dir + trajectory_tag
    t_arr, r_arr, x_arr, segments = get_pickle_trajectory(path_to_file)
    t_segs, x_segs = get_stay_segs(get_adjusted_stays(segments, t_arr))

    # Get the stay clusters
    all_clusters = stay_classifier_testing(t_arr, x_arr, dist_thresh, time_thresh, verbose)    
    
    stage_titels = [
            f"Stage {1}: get mini-clusters and merging",
            f"Stage {2}: extend clusters and IQR-filter",
            f"Stage {3}: separate overlapping clusters",
            f"Stage {4}: merge nearby clusters",
            f"Stage {5}: shift the boxes",
            f"Stage {6}: filter regions by IQR"
        ]

    final_len=len(all_clusters[-1])
    prec, rec, conmat = eval_synth_data(segments, t_arr, all_clusters[-1])
    
    results_dir = "good/"
    
    staystr = f"{final_len}"
    precstr = f"{prec:6.3f}".strip().replace(".","o")
    recstr = f"{rec:6.3f}".strip().replace(".","o")

    stays_tag = int((x_segs.size)/3)
    notes = f"{staystr}stays_prec{precstr}_rec{recstr}"

    trajectory_tag = f"trajectory{ii}_{stays_tag}stays" 
    trajectory_tag += f"__{notes}"
    
    if (prec < precrec_limit) | (rec < precrec_limit) | (final_len != stays_tag):    
        #if ((prec < precrec_limit) | (rec < precrec_limit)) & (final_len == 3):
        reason = f"Traj.{ii}: "
        if (final_len != stays_tag): 
            reason += " wrong stays;"
        if (prec < precrec_limit):
            reason += " p low;"
        if (rec < precrec_limit):
            reason += " r low;"
        
        #print(ii, reason)        
        bad_list.append(trajectory_tag)
        results_dir = "bad/"
    else:
        results_dir = "good/"
    
    len_all_clusts = len(all_clusters)
    
    fig, axs = plt.subplots(len_all_clusts,1,figsize=[20,len_all_clusts*6])
    fig.subplots_adjust(bottom=0.1, top=0.9, hspace=0.4)
    ylim = [x_arr.min()-2*dist_thresh, x_arr.max()+2*dist_thresh]

    axs =axs.flatten()

    for i in range(len_all_clusts):
        try:
            ax = axs[i]
            add_plot_trajectory(t_arr, r_arr, x_arr, t_segs, x_segs, dist_thresh, ax)
            #ax.set_xlim(0.7,23.3)
            ax.set_xlim([5.75,18.25]);
            add_plot_seg_boxes(t_segs, x_segs, dist_thresh, ax)
            add_plot_cluster_boxplots(t_arr, x_arr, all_clusters[-1], dist_thresh, ax)
            ax.set_ylim(ylim);
            ax.set_title(f"{stage_titels[-1]}", fontsize=16);
        except:
            print(f"File {ii}, plotting stage {i} failed")
            pass

    fig.suptitle(f"{final_len:4d} Stays, prec. {prec:6.3f}, {rec:6.3f}", fontsize=16)
    
    path_to_file =  data_dir_plots + results_dir + trajectory_tag
    plt.savefig(path_to_file + ".png")
    plt.close()
    
    if final_len != stays_tag:
        lens.append(final_len)
        precs.append(prec)
        recs.append(rec)
    else:
        lens3.append(final_len)
        precs3.append(prec)
        recs3.append(rec)
    
    
    if ii % int(0.1*total) == 0:
        print(f"{ii:4d} of {total:5d}")


 800 of  1000
 900 of  1000
